In [16]:
import os
import pandas as pd
import logging
import re
from scripts.preprocesar_asistencias_girardot import procesar_excel
from scripts.preprocesar_asistencias_fusagasuga import consolidar_archivos_fusagasuga_csv ,consolidar_asistencia_fusagasuga
from scripts.preprocesar_asistencias_fusagasuga import quitar_tildes,limpiar_espacios,convertir_a_formato_largo
from scripts.preprocesar_asistencias_fusagasuga import convertir_a_numero_mes_grado,agregar_bimestre_con_dias,mapear_estado_asistencia

In [17]:
# Ruta del archivo Excel
ruta_archivo = 'Asistencias Girardot 2024/Registro de asistencia 2024-Estudiantes.xlsx'

# Llamada a la función
df_resultado = procesar_excel(ruta_archivo)

# Vista preliminar del DataFrame resultante
df_resultado.head()



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Hoja,ccc
0,COD.,NOMBRES Y APELLIDOS ALUMNO,Mes,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,Prejardín,NaN
1,J-01,MARGARET ACOSTA SEGURA,Febrero,X,-,NaN,NaN,-,X,X,...,NaN,NaN,X,X,X,X,X,X,Prejardín,NaN
2,J-02,ANA VERONICA SUAREZ CASASBUENAS,Febrero,-,-,NaN,NaN,X,X,X,...,NaN,NaN,X,X,X,X,X,X,Prejardín,NaN
3,J-03,MIA CASTAÑEDA SANTOS,Febrero,X,X,NaN,NaN,X,X,E,...,NaN,NaN,X,X,X,X,X,X,Prejardín,NaN
4,COD.,NOMBRES Y APELLIDOS ALUMNO,Mes,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,Prejardín,NaN


In [18]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   302 non-null    object
 1   Unnamed: 1   730 non-null    object
 2   Unnamed: 2   729 non-null    object
 3   Unnamed: 3   448 non-null    object
 4   Unnamed: 4   386 non-null    object
 5   Unnamed: 5   310 non-null    object
 6   Unnamed: 6   351 non-null    object
 7   Unnamed: 7   489 non-null    object
 8   Unnamed: 8   454 non-null    object
 9   Unnamed: 9   471 non-null    object
 10  Unnamed: 10  610 non-null    object
 11  Unnamed: 11  567 non-null    object
 12  Unnamed: 12  453 non-null    object
 13  Unnamed: 13  459 non-null    object
 14  Unnamed: 14  609 non-null    object
 15  Unnamed: 15  351 non-null    object
 16  Unnamed: 16  335 non-null    object
 17  Unnamed: 17  728 non-null    object
 18  Unnamed: 18  506 non-null    object
 19  Unnamed: 19  454 non-null    

In [19]:
df_resultado.columns = ["cod"	,"NOMBRES Y APELLIDOS ALUMNO"	,"Mes",
                        	"1","2","3","4","5","6","7","8","9","10",
                            "11","12","13","14","15","16","17","18","19","20",
                            "21","22","23","24","25","26","27","28","29","30","31",                          
                        "Grado","ccc"
                            	]


In [20]:
asistencia_df_preprocesada = df_resultado.copy()
asistencia_df_preprocesada = asistencia_df_preprocesada[asistencia_df_preprocesada["cod"] != 'COD.']
asistencia_df_preprocesada = asistencia_df_preprocesada[asistencia_df_preprocesada["ccc"] != 'COD.']
asistencia_df_preprocesada = asistencia_df_preprocesada[asistencia_df_preprocesada["Mes"] != 'Mes']
asistencia_df_preprocesada = asistencia_df_preprocesada.drop(columns=['cod', 'ccc'])
asistencia_df_preprocesada=asistencia_df_preprocesada.fillna("D")
asistencia_df_preprocesada["estudiante"]=asistencia_df_preprocesada["NOMBRES Y APELLIDOS ALUMNO"].apply(quitar_tildes)
asistencia_df_preprocesada["estudiante"]=asistencia_df_preprocesada["estudiante"].apply(limpiar_espacios)
asistencia_df_preprocesada.drop("NOMBRES Y APELLIDOS ALUMNO", axis=1, inplace=True)
asistencia_df_preprocesada["Sede"]="Girardot"
asistencia_df_preprocesada["Año"]="2024"


In [21]:

asistencia_df_preprocesada.to_csv('pre-asistencia_girardot_2024_horizontal.csv', index=False)

In [22]:
# Reestructurar al formato largo
asistencia_df_preprocesada["estudiante"]=asistencia_df_preprocesada["estudiante"].str.upper()
id_vars = ["Mes", "Grado", "Año", "estudiante","Sede"]
df_consolidado_largo= convertir_a_formato_largo(asistencia_df_preprocesada,id_vars,1,31)

In [23]:
df_consolidado_largo=convertir_a_numero_mes_grado(df_consolidado_largo)
df_consolidado_largo=agregar_bimestre_con_dias(df_consolidado_largo)
df_consolidado_largo.to_csv('pre-asistencia_girardot_2024_vertical.csv', index=False)

In [24]:
df_consolidado_largo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20243 entries, 0 to 20242
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Mes         20243 non-null  object
 1   Grado       20243 non-null  object
 2   Año         20243 non-null  object
 3   estudiante  20243 non-null  object
 4   Sede        20243 non-null  object
 5   Día         20243 non-null  int64 
 6   Estado      20243 non-null  object
 7   Mes_Num     20243 non-null  int64 
 8   Grado_Num   20243 non-null  int64 
 9   Bimestre    20243 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.5+ MB


In [25]:
df_estudiantes = pd.read_csv( "notas/Estudiantes.csv", delimiter=";")
df_estudiantes['estudiante2'] = df_estudiantes['Nombres'].str.cat(
    [df_estudiantes['Primer Apellido'], df_estudiantes['Segundo Apellido']], sep=" ", na_rep=""
)
df_estudiantes.head()

,Id,Sede,Nivel,Orden Grado,Grado,Antigüedad,Género,Documento de Identificación,Primer Apellido,Segundo Apellido,Nombres,Fecha de nacimiento,estudiante,estudiante2
0,1,Girardot,Preescolar,-3,Prejardín,Nuevo,Femenino,1233700469,ACOSTA,SEGURA,MARGARET,11/4/2021,ACOSTA SEGURA MARGARET,MARGARET ACOSTA SEGURA
1,2,Girardot,Preescolar,-3,Prejardín,Nuevo,Femenino,1015000870,SUAREZ,CASASBUENAS,ANA VERONICA,18/2/2021,SUAREZ CASASBUENAS ANA VERONCA,ANA VERONICA SUAREZ CASASBUENAS
2,3,Girardot,Preescolar,-3,Prejardín,Antiguo,Masculino,1071063283,RODRIGUEZ,SALCEDO,JACOBO,6/5/2020,RODRIGUEZ SALCEDO JACOBO,JACOBO RODRIGUEZ SALCEDO
3,4,Girardot,Preescolar,-3,Prejardín,Nuevo,Femenino,1012926444,CASTAÑEDA,SANTOS,MIA ISABEL,1/7/2020,CASTAÑEDA SANTOS MIA ISABEL,MIA ISABEL CASTAÑEDA SANTOS
4,5,Girardot,Preescolar,-2,Jardín,Antiguo,Femenino,1070630315,LUNA,GARCIA,ANTONELLA,5/11/2019,LUNA GARCIA ANTONELLA,ANTONELLA LUNA GARCIA


In [26]:
df_estudiantes["estudiante2"]=df_estudiantes["estudiante2"].apply(quitar_tildes)
df_estudiantes["estudiante2"]=df_estudiantes["estudiante2"].apply(limpiar_espacios)

In [27]:
# Realizar el merge
df_consolidado_largo = df_consolidado_largo.rename(columns={'estudiante': 'estudiante2'})
df_merge = pd.merge(df_consolidado_largo, df_estudiantes, on="estudiante2", how="left")

In [28]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20243 entries, 0 to 20242
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Mes                          20243 non-null  object 
 1   Grado_x                      20243 non-null  object 
 2   Año                          20243 non-null  object 
 3   estudiante2                  20243 non-null  object 
 4   Sede_x                       20243 non-null  object 
 5   Día                          20243 non-null  int64  
 6   Estado                       20243 non-null  object 
 7   Mes_Num                      20243 non-null  int64  
 8   Grado_Num                    20243 non-null  int64  
 9   Bimestre                     20243 non-null  object 
 10  Id                           16120 non-null  float64
 11  Sede_y                       16120 non-null  object 
 12  Nivel                        16120 non-null  object 
 13  Orden Grado     

In [15]:
df_merge.to_csv('asistencia_giradot_2024_con_doc.csv', index=False)

PENDIENTE COMPLETAR NOMBRES CON DOCUMENTOS, LOS DOS CSV TIENEN VALORES DISTINTOS